<a href="https://colab.research.google.com/github/merttguney/AI_106/blob/main/Gorev15_Mert_G%C3%BCney.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Türkçe veri seti bulun ve Türkçe duygu analizi yapın**

In [16]:
# Kütüphane olarak yapay zekadan tavsiye aldım. İşlemediğimiz kütüphane metodlarını ona kullandırttım. Geliştirdim modelimi.
!pip install TurkishStemmer -q
!pip install imbalanced-learn -q

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from TurkishStemmer import TurkishStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [19]:
df = pd.read_csv("magaza_yorumlari_duygu_analizi.csv", encoding="utf-16")

In [17]:
df = df.dropna(subset=['Görüş', 'Durum'])

In [20]:
df.head()

,Görüş,Durum
0,"ses kalitesi ve ergonomisi rezalet, sony olduğ...",Olumsuz
1,hizli teslimat tesekkürler,Tarafsız
2,ses olayı süper....gece çalıştır sıkıntı yok.....,Olumlu
3,geldi bigün kullandık hemen bozoldu hiçtavsiye...,Olumsuz
4,Kulaklığın sesi kaliteli falan değil. Aleti öv...,Olumsuz


In [3]:
def clean_text(text):
    if isinstance(text, float):
        text = str(text)
    text = text.lower()
    text = re.sub(r'[^a-zçğıöşü ]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

stemmer = TurkishStemmer()
tr_stopwords = stopwords.words('turkish')

def preprocess(text):
    if pd.isna(text):
        text = ""
    text = clean_text(text)
    words = text.split()
    words = [stemmer.stem(w) for w in words if w not in tr_stopwords and len(w) > 2]
    return ' '.join(words)

df["Processed"] = df["Görüş"].apply(preprocess)

In [4]:
tfidf = TfidfVectorizer(max_features=5000, ngram_range=(1, 2), stop_words=tr_stopwords)
X = tfidf.fit_transform(df["Processed"])
y = df["Durum"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# SMOTE ile azınlık sınıfları çoğaltma işlemi(Chatgpt önerisi ile)
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)

In [6]:
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_res, y_res)

LogisticRegression(max_iter=1000, random_state=42)

In [21]:
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.6660833041652082

Classification Report:
               precision    recall  f1-score   support

      Olumlu       0.78      0.78      0.78      1073
     Olumsuz       0.71      0.70      0.71      1045
    Tarafsız       0.44      0.46      0.45       739

    accuracy                           0.67      2857
   macro avg       0.65      0.64      0.65      2857
weighted avg       0.67      0.67      0.67      2857



In [8]:
def duygu_tahmini(cumle):
    processed = preprocess(cumle)
    vec = tfidf.transform([processed])
    return model.predict(vec)[0]

In [15]:
print("Ürün mükemmel, herkese tavsiye ederim : ", duygu_tahmini("Ürün mükemmel, herkese tavsiye ederim"))
print("Fena değildi ama daha iyi olabilirdi : ", duygu_tahmini("Fena değildi ama daha iyi olabilirdi"))
print("Kargo hızlıydı, paketleme güzeldi : ", duygu_tahmini("Kargo hızlıydı, paketleme güzeldi"))
print("Berbat bir müşteri hizmeti : ", duygu_tahmini("Berbat bir müşteri hizmeti"))

Ürün mükemmel, herkese tavsiye ederim :  Olumlu
Fena değildi ama daha iyi olabilirdi :  Tarafsız
Kargo hızlıydı, paketleme güzeldi :  Olumlu
Berbat bir müşteri hizmeti :  Olumsuz


In [22]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
model_proba = LogisticRegression(max_iter=1000, random_state=42)
y_encoded = le.fit_transform(y_res)
model_proba.fit(X_res, y_encoded)

def duygu_tahmini_skorlu(cumle):
    processed = preprocess(cumle)
    vec = tfidf.transform([processed])
    pred = model_proba.predict(vec)[0]
    prob = model_proba.predict_proba(vec)[0][pred]
    return le.inverse_transform([pred])[0], prob

# Örnek bu şekilde
cumle = "Çok kötü bir alışverişti, asla tavsiye etmem"
etiket, skor = duygu_tahmini_skorlu(cumle)
print(f"'{cumle}'  {etiket} ({skor:.2f})")

'Çok kötü bir alışverişti, asla tavsiye etmem'  Olumsuz (0.90)
